In [82]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.expand_frame_repr", False)


In [83]:
df_ = pd.read_excel(r"D:\Data_Science_Path\Recommendation_Systems\notebook\datasets\online_retail_II.xlsx", sheet_name="Year 2010-2011", engine= "openpyxl")

In [84]:
df = df_.copy()

In [85]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [86]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,541910.0,9.552234,-80995.0,1.0,3.0,10.0,80995.0,218.080957
InvoiceDate,541910,2011-07-04 13:35:22.342307584,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
Price,541910.0,4.611138,-11062.06,1.25,2.08,4.13,38970.0,96.759765
Customer ID,406830.0,15287.68416,12346.0,13953.0,15152.0,16791.0,18287.0,1713.603074


The minumum value of Quantity and price is negative values. This is nonsense. Check this out.

In [87]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [88]:
df.dropna(inplace=True)

In [89]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [90]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [91]:
df = df[df["Quantity"] > 0]

In [92]:
df = df[df["Price"]>0]

In [93]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397885.0,12.988208,1.0,2.0,6.0,12.0,80995.0,179.331551
InvoiceDate,397885,2011-07-10 23:41:56.419316992,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
Price,397885.0,3.116525,0.001,1.25,1.95,3.75,8142.75,22.097861
Customer ID,397885.0,15294.416882,12346.0,13969.0,15159.0,16795.0,18287.0,1713.144421


We handled negative values and missing values. But I observe that we have outliers. Looking at the Quantity 75% to max value. Jumping 12.0 to 80995 and Price variable too.

## Outliers

In [94]:
def outlier_threshold(dataframe, variable):
    quartile_1 =dataframe[variable].quantile(0.01)
    quartile_3 =dataframe[variable].quantile(0.99)
    IQR = quartile_3 - quartile_1
    up_limit = quartile_3 + IQR * 1.5
    low_limit = quartile_1 - IQR * 1.5
    return low_limit, up_limit

In [95]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_threshold(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [96]:
replace_with_thresholds(df,"Quantity")

In [97]:
replace_with_thresholds(df,"Price")

In [98]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397885.0,11.83077,1.0,2.0,6.0,12.0,298.5,25.523052
InvoiceDate,397885,2011-07-10 23:41:56.419316992,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
Price,397885.0,2.893492,0.001,1.25,1.95,3.75,37.06,3.227175
Customer ID,397885.0,15294.416882,12346.0,13969.0,15159.0,16795.0,18287.0,1713.144421


Prepare Data structure

In [99]:
df_fr = df[df["Country"] == "France"]

In [100]:
df_fr.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24.0,2010-12-01 08:45:00,3.75,12583.0,France
27,536370,22727,ALARM CLOCK BAKELIKE RED,24.0,2010-12-01 08:45:00,3.75,12583.0,France
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12.0,2010-12-01 08:45:00,3.75,12583.0,France
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12.0,2010-12-01 08:45:00,0.85,12583.0,France
30,536370,21883,STARS GIFT TAPE,24.0,2010-12-01 08:45:00,0.65,12583.0,France


In [101]:
df_fr.groupby(["Invoice","Description"]).agg({"Quantity":"sum"}).head(25)

Quantity
Invoice Description                                  
536370   SET 2 TEA TOWELS I LOVE LONDON          24.0
        ALARM CLOCK BAKELIKE GREEN               12.0
        ALARM CLOCK BAKELIKE PINK                24.0
        ALARM CLOCK BAKELIKE RED                 24.0
        CHARLOTTE BAG DOLLY GIRL DESIGN          20.0
        CIRCUS PARADE LUNCH BOX                  24.0
        INFLATABLE POLITICAL GLOBE               48.0
        LUNCH BOX I LOVE LONDON                  24.0
        MINI JIGSAW CIRCUS PARADE                24.0
        MINI JIGSAW SPACEBOY                     24.0
        MINI PAINT SET VINTAGE                   36.0
        PANDA AND BUNNIES STICKER SHEET          12.0
        POSTAGE                                   3.0
        RED TOADSTOOL LED NIGHT LIGHT            24.0
        ROUND SNACK BOXES SET OF4 WOODLAND       24.0
        SET/2 RED RETROSPOT TEA TOWELS           18.0
        SPACEBOY LUNCH BOX                       24.0
        STARS GIFT TAPE                          24.0
        VINTAGE HEADS AND TAILS CARD GAME        24.0
        VINTAGE SEASIDE JIGSAW PUZZLES           12.0
536852  CHARLOTTE BAG DOLLY GIRL DESIGN          10.0
        MINI JIGSAW DOLLY GIRL                   24.0
        MINI JIGSAW SPACEBOY                     24.0
        PICTURE DOMINOES                         12.0
        POLKADOT RAIN HAT                        24.0

In [102]:
df_fr.groupby(["Invoice","Description"]).agg({"Quantity":"sum"}).unstack().iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                 NaN                NaN                          NaN                      NaN                             24.0
536852                                 NaN                NaN                          NaN                      NaN                              NaN
536974                                 NaN                NaN                          NaN                      NaN                              NaN
537065                                 NaN                NaN                          NaN                      NaN                              NaN
537463                                 NaN                NaN                          NaN                      NaN                              NaN

In [103]:
df_fr.groupby(["Invoice","Description"]).agg({"Quantity":"sum"}).unstack().fillna(0).iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                 0.0                0.0                          0.0                      0.0                             24.0
536852                                 0.0                0.0                          0.0                      0.0                              0.0
536974                                 0.0                0.0                          0.0                      0.0                              0.0
537065                                 0.0                0.0                          0.0                      0.0                              0.0
537463                                 0.0                0.0                          0.0                      0.0                              0.0

In [54]:
df_fr.groupby(["Invoice","Description"]).agg({"Quantity":"sum"}).unstack().fillna(0).applymap(lambda x : 1 if x>0 else 0).iloc[0:5,0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                   0                  0                            0                        0                                1
536852                                   0                  0                            0                        0                                0
536974                                   0                  0                            0                        0                                0
537065                                   0                  0                            0                        0                                0
537463                                   0                  0                            0                        0                                0

In [104]:
df_fr.groupby(["Invoice","StockCode"]).agg({"Quantity":"sum"}).unstack().fillna(0).applymap(lambda x : 1 if x>0 else 0)

Quantity                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [111]:
def create_invoice_product_df(dataframe, id = False):
    if id:
        return dataframe.groupby(["Invoice","StockCode"])["Quantity"].sum().unstack().fillna(0).applymap(lambda x : 1 if x > 0 else 0)

    else:
         return dataframe.groupby(["Invoice","Description"])["Quantity"].sum().unstack().fillna(0).applymap(lambda x : 1 if x > 0 else 0)

In [112]:
france_inv_df = create_invoice_product_df(df_fr, id=True)

In [114]:
france_inv_df.shape

(389, 1522)

In [115]:
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"]== stock_code][["Description"]].values[0].tolist()
    print(product_name)

In [117]:
check_id(df_fr, 10135)

['COLOURING PENCILS BROWN TUBE']


Association Rules Analysis

In [127]:
frquent_itemsets = apriori(france_inv_df, 
                           min_support = 0.015, 
                           use_colnames=True)

d:\Data_Science_Path\Recommendation_Systems\venv\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


I have memory error because of min_support = 0.01. So change it to 0.02

In [128]:
frquent_itemsets.sort_values("support", ascending=False)

,support,itemsets
377,0.773779,(POST)
275,0.187661,(23084)
82,0.179949,(21731)
177,0.172237,(22554)
179,0.169666,(22556)
...,...,...
4021,0.015424,"(22553, 22556, 22551)"
4022,0.015424,"(POST, 22553, 22551)"
1210,0.015424,"(23084, 22029)"
1209,0.015424,"(23076, 22029)"


In [129]:
rules = association_rules(frquent_itemsets, metric="support", min_threshold=0.01)

In [130]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(POST),(10002),0.773779,0.020566,0.015424,0.019934,0.969269,-0.000489,0.999355,-0.122924
1,(10002),(POST),0.020566,0.773779,0.015424,0.750000,0.969269,-0.000489,0.904884,-0.031356
2,(20725),(10125),0.154242,0.015424,0.015424,0.100000,6.483333,0.013045,1.093973,1.000000
3,(10125),(20725),0.015424,0.154242,0.015424,1.000000,6.483333,0.013045,inf,0.859008
4,(20675),(20676),0.015424,0.038560,0.015424,1.000000,25.933333,0.014829,inf,0.976501
...,...,...,...,...,...,...,...,...,...,...
56019,(21094),"(21987, 21988, 21989, 22435, POST, 21080, 21086)",0.128535,0.015424,0.015424,0.120000,7.780000,0.013442,1.118836,1.000000
56020,(22435),"(21987, 21988, 21989, 21094, POST, 21080, 21086)",0.066838,0.025707,0.015424,0.230769,8.976923,0.013706,1.266581,0.952250
56021,(POST),"(21987, 21988, 21989, 21094, 22435, 21080, 21086)",0.773779,0.015424,0.015424,0.019934,1.292359,0.003489,1.004601,1.000000
56022,(21080),"(21987, 21988, 21989, 21094, 22435, POST, 21086)",0.133676,0.015424,0.015424,0.115385,7.480769,0.013362,1.112999,1.000000


In [131]:
rules[(rules["support"]>0.05) & (rules["confidence"] >0.01) & (rules["lift"]>5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
714,(21080),(21086),0.133676,0.138817,0.102828,0.769231,5.541311,0.084271,3.731791,0.945994
715,(21086),(21080),0.138817,0.133676,0.102828,0.740741,5.541311,0.084271,3.341535,0.951642
716,(21080),(21094),0.133676,0.128535,0.102828,0.769231,5.984615,0.085646,3.776350,0.961424
717,(21094),(21080),0.128535,0.133676,0.102828,0.800000,5.984615,0.085646,4.331620,0.955752
806,(21086),(21094),0.138817,0.128535,0.123393,0.888889,6.915556,0.105550,7.843188,0.993284
...,...,...,...,...,...,...,...,...,...,...
35918,"(POST, 22727)","(22728, 22726)",0.089974,0.074550,0.059126,0.657143,8.814778,0.052418,2.699229,0.974208
35919,"(22726, 22727)","(22728, POST)",0.079692,0.092545,0.059126,0.741935,8.017025,0.051751,3.516388,0.951057
35920,(22728),"(POST, 22726, 22727)",0.102828,0.074550,0.059126,0.575000,7.712931,0.051460,2.177529,0.970101
35922,(22726),"(22728, POST, 22727)",0.097686,0.069409,0.059126,0.605263,8.720273,0.052346,2.357498,0.981172


In [133]:
rules[(rules["support"]>0.05) & (rules["confidence"] >0.01) & (rules["lift"]>5)].sort_values("confidence", ascending= False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
7007,"(21080, 21094)",(21086),0.102828,0.138817,0.100257,0.975000,7.023611,0.085983,34.447301,0.955918
7006,"(21080, 21086)",(21094),0.102828,0.128535,0.100257,0.975000,7.585500,0.087040,34.858612,0.967673
22694,"(21080, POST, 21086)",(21094),0.084833,0.128535,0.082262,0.969697,7.544242,0.071358,28.758355,0.947858
22695,"(21080, POST, 21094)",(21086),0.084833,0.138817,0.082262,0.969697,6.985410,0.070486,28.419023,0.936271
807,(21094),(21086),0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563,0.981563
...,...,...,...,...,...,...,...,...,...,...
2876,(22629),(22630),0.125964,0.100257,0.071979,0.571429,5.699634,0.059351,2.099400,0.943382
16359,(22630),"(POST, 22629)",0.100257,0.100257,0.053985,0.538462,5.370809,0.043933,1.949443,0.904490
16354,"(POST, 22629)",(22630),0.100257,0.100257,0.053985,0.538462,5.370809,0.043933,1.949443,0.904490
16358,(22629),"(POST, 22630)",0.125964,0.074550,0.053985,0.428571,5.748768,0.044594,1.619537,0.945098


Recommend İtem

Ex : Customer choose item ID: 22492

In [134]:
product_id = 22492

In [135]:
check_id(df, product_id)

['MINI PAINT SET VINTAGE ']


In [136]:
sorted_rules = rules.sort_values("lift", ascending= False)

In [137]:
recommendation_list = []

In [140]:
for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

In [144]:
check_id(df,recommendation_list[0])

['SET/20 RED RETROSPOT PAPER NAPKINS ']
